Rock - Paper - Scissors CNN Classifier

In [ ]:

import json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, GlobalAveragePooling2D
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report


In [ ]:
seed = 17
image_size = (150, 150)
batch_size = 710
classes = ("paper", "rock", "scissors")
validation_split = 0.2

Preprocessing
Explore the dataset thoroughly and provide a summary of your observations.
Perform necessary preprocessing steps:
    - [x] Explore and plot the data
    - [x] Image resizing.
    - [x] Image normalization.
    - [x] Optionally, data augmentation techniques.
    - [x] Splitting the data into training and test sets appropriately.

In [ ]:
initial_train_ds = tf.keras.utils.image_dataset_from_directory(
    "data_reconstructed/train",
    image_size=image_size,
    shuffle=True,
    seed=seed,
    
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "data_reconstructed/validation",
    image_size=image_size,
    shuffle=False,
    seed=seed,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "data_reconstructed/test",
    image_size=image_size,
    shuffle=False,
    seed=seed,
)

In [ ]:
def visualize_dataset(train_ds, classes):
    plt.figure(figsize=(10, 10))
    for images, labels in train_ds.take(1):
        for i in range(12):
            plt.subplot(4, 3, i + 1)
            plt.imshow(np.array(images[i]).astype("uint8"))
            plt.title(classes[int(labels[i])])
            plt.axis("off")

In [ ]:
visualize_dataset(initial_train_ds, classes)

In [ ]:
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical", seed=seed),
    layers.RandomBrightness(0.2, seed=seed),
    layers.RandomZoom(0.2, seed=seed),
    layers.RandomContrast(0.4, seed=seed),
    layers.RandomCrop(height=image_size[0], width=image_size[1], seed=seed),
])

augmented_train_ds = initial_train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))


In [ ]:
visualize_dataset(augmented_train_ds, classes)

### Image normalization

In [ ]:
# normalization
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_ds = augmented_train_ds.map(normalize)
val_ds = val_ds.map(normalize)
test_ds = test_ds.map(normalize)

### Convert dataset to numpy array and split into feature matrix and label vector

In [ ]:

def dataset_to_numpy(ds):
    X, y = [], []
    for images, labels in ds:
        X.append(images.numpy())
        y.append(labels.numpy())
    return tf.concat(X, axis=0), tf.concat(y, axis=0)

In [ ]:
X_train, y_train = dataset_to_numpy(train_ds)
X_validation, y_validation = dataset_to_numpy(val_ds)
X_test, y_test = dataset_to_numpy(test_ds)

In [ ]:

print(np.unique(y_train,return_counts=True),np.unique(y_validation,return_counts=True), np.unique(y_test,return_counts=True))

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

y_train_np = y_train.numpy() if hasattr(y_train, 'numpy') else np.array(y_train)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_np),
    y=y_train_np
)
class_weights_dict = dict(enumerate(class_weights))
print(class_weights_dict)

The dataset is not imbalanced since the differences between the occurance of classes is not big.

Simple model
- one convolutional layer, 32 fiters 3x3 grid
- maxpooling 2x2
- hidden layer NN with 512 neurons
- softmax

In [ ]:
def build_simple_model(hp):
    model = Sequential()
    model.add(Input(shape=(image_size[0], image_size[1], 3)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(Conv2D(
        filters=hp.Int('conv_filters', min_value=32, max_value=128, step=32),
        kernel_size=3,
        activation='relu',
    ))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense( 
        units = hp.Int('dense_units', min_value=128, max_value=512, step=128), 
        activation='relu'))
    model.add(Dense(units=3, activation='softmax'))
    
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4], default=1e-3)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

### Tuning the model
Tuning params
 - Number of filters for the Convolutional Layer
 - Number of units for the densely-connected NN layer
 - Learning rate for the Optimizer

In [ ]:
import keras_tuner as kt

tuner = kt.Hyperband(
    build_simple_model,
    objective='val_accuracy',
    max_epochs=30,
    directory='tuning',
    project_name='cnn_tuning'
)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)


In [ ]:

tuner.search(train_ds, validation_data=val_ds, epochs=30, callbacks=[stop_early])

In [ ]:
best_model_1 = tuner.get_best_models(num_models=1)[0]
best_hyperparams = tuner.get_best_hyperparameters(1)[0]
best_learning_rate = best_hyperparams.get('learning_rate')
best_conv_filters = best_hyperparams.get('conv_filters')
best_dense_units = best_hyperparams.get('dense_units')
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]
best_epoch = best_trial.hyperparameters.get('tuner/epochs')

print("Best Conv filters:", best_hyperparams.get('conv_filters'))
print("Best Dense units:", best_hyperparams.get('dense_units'))
print("Best Learning rate:", best_hyperparams.get('learning_rate'))
print("Best Epoch:", best_epoch)

best_model_1.summary()

### Manually tuning the batch size on the best parameters model

In [ ]:
import numpy as np
batch_sizes = [16, 32, 64]  
results = []

for batch_size in batch_sizes:
    print(f"Training with batch size: {batch_size}")
    
    model = best_model_1
    
    history = model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_data=(X_validation, y_validation), callbacks=[stop_early])

    val_accuracy = history.history['val_accuracy'][-1]
    results.append((batch_size, val_accuracy))
    print(f"Validation Accuracy for batch size {batch_size}: {val_accuracy}")

results = sorted(results, key=lambda x: x[1], reverse=True)

best_batch_size, best_val_accuracy = results[0]
print(f"Best Batch Size: {best_batch_size}, Best Validation Accuracy: {best_val_accuracy}")


### Used tuned params 

In [ ]:
best_model_1.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=best_learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
best_model_1.summary()

In [ ]:

from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(
    filepath='best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min'
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

simple_history = best_model_1.fit(
    X_train,
    y_train,
    validation_data=(X_validation, y_validation),
    epochs=5,
    callbacks=[early_stopping, checkpointer],
    batch_size=32
).history


In [ ]:
def plot_history(history):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history['accuracy'], label='Train Accuracy')
    plt.plot(history['val_accuracy'], label='Val Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(history['loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

### Simple Model Results

In [ ]:
plot_history(simple_history)

### Provide visualizations of training curves (loss and accuracy)

In [ ]:
model_1_improvement = Sequential()
model_1_improvement.add(Input(shape=(image_size[0], image_size[1], 3)))
model_1_improvement.add(Conv2D(32, (3,3), activation='relu'))
model_1_improvement.add(MaxPooling2D(2,2))
model_1_improvement.add(Dropout(0.15))
model_1_improvement.add(Flatten())
model_1_improvement.add(Dense(512, activation='relu'))
model_1_improvement.add(Dropout(0.3))
model_1_improvement.add(Dense(3, activation='softmax'))

model_1_improvement.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=best_learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model_1_improvement.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history_improvement = model_1_improvement.fit(
    X_train,
    y_train,
    validation_data=(X_validation, y_validation),
    epochs=20,
    callbacks=[early_stopping],
    batch_size=32, 
    class_weight=class_weights_dict
)

### Improvement Results

In [ ]:
plot_history(history_improvement)

In [ ]:
def predict(model, X_test):
    y_pred = model.predict(X_test)
    pred = np.argmax(y_pred, axis=1)
    return pred

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.show()

### Confusion Matrix
#### Simple Best Model Results

In [ ]:
pred = predict(best_model_1, X_test)
plot_confusion_matrix(y_test, pred, classes)
print(classification_report(y_test,pred,target_names = classes, digits=7))

#### Improved Simple Model Results

In [ ]:
pred_improvement = predict(model_1_improvement, X_test)
print(classification_report(y_test,pred_improvement,target_names = classes, digits=7))
plot_confusion_matrix(y_test, pred_improvement, classes)

### Making a more complex model

In [ ]:
model_2 = Sequential()
model_2.add(Input(shape=(150, 150, 3)))

model_2.add(Conv2D(32, (3, 3), activation='relu'))
model_2.add(MaxPooling2D(2, 2))
model_2.add(Dropout(0.1))

model_2.add(Conv2D(64, (3, 3), activation='relu'))
model_2.add(MaxPooling2D(2, 2))
model_2.add(Dropout(0.1))

model_2.add(Flatten())
model_2.add(Dense(units=128, activation='relu'))
model_2.add(Dropout(0.1))
model_2.add(Dense(units=512, activation='relu'))
model_2.add(Dropout(0.1))
model_2.add(Dense(units=3, activation='softmax'))



model_2.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model_2.summary()


In [ ]:

from keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stop = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                               factor=0.1, 
                               patience=2, 
                               min_lr=0.000001)
history_2 = model_2.fit(
    X_train,
    y_train,
    validation_data=(X_validation, y_validation),
    epochs=10,
    batch_size=32,
    class_weight=class_weights_dict,
    callbacks=[reduce_lr, early_stop]
)

In [ ]:
plot_history(history_2)

In [ ]:
y_pred = model_2.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

plot_confusion_matrix(y_test, y_pred_classes, classes)
print(classification_report(y_test, y_pred_classes, digits=7, target_names=classes))

### Check generalization on the More Complex Model on Personal Data

In [ ]:
generalization_ds = tf.keras.utils.image_dataset_from_directory(
    "generalization_bg_removed",
    image_size=image_size,
    shuffle=False,
    seed=12,
)

# rotate the images because the pictures are 300x200 instead of 200x300
def rotate_image(image, label):
    image = tf.image.rot90(image, k=1)
    return image, label
generalization_ds = generalization_ds.map(rotate_image)


In [ ]:
for images, labels in generalization_ds.take(1):
    for i in range(3):
        plt.subplot(4, 3, i + 1)
        plt.imshow(np.array(images[i]).astype("uint8"))
        

normalized_ds = generalization_ds.map(normalize)
X_generalization, y_generalization = dataset_to_numpy(normalized_ds)

y_pred = model_2.predict(X_generalization)
y_pred_classes = np.argmax(y_pred, axis=1)

plot_confusion_matrix(y_generalization, y_pred_classes, classes)
print(classification_report(y_generalization, y_pred_classes, digits=7, target_names=classes))